# Análise de sentimentos na base do IMDB usando BoW e LORA

Leandro Carísio Fernandes

# Parâmetros

In [53]:
MAX_VOCAB_SIZE = 20000  # Máximo de palavras no vocabulário
VOCAB_SIZE = -1         # Será ajustado automaticamente depois do cálculo do vocabulário

HIDDEN_SIZE = 200       # Tamanho da camada oculta

LR = 0.1                # Learning rate
NUM_EPOCHS = 7          # Número de épocas para rodar o treinamento completo. Após 7 épocas já não tem melhoras significativas no dataset de validação
NUM_EPOCHS_LORA = 10     # Número de épocas para rodar fine-tuning com LoRA (será executado após apenas uma época de treinamento completo)
RANK = 1                # Rank que será usado. Quanto menor, menos parâmetros serão treinados no fine-tuning
ALPHA = 1               # Usado pro cálculo do fator de escala na matriz delta_w. Escala = alpha/rank

In [54]:
import torch

# Verifica se há uma GPU disponível e define o dispositivo para GPU se possível, caso contrário, usa a CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
if device.type == 'cuda':
    print('GPU:', torch.cuda.get_device_name(torch.cuda.current_device()))
else:
    print('using CPU')

GPU: Tesla T4


# Seed

In [55]:
import random
import torch.nn.functional as F
import numpy as np

def inicializa_seeds():
  random.seed(123)
  np.random.seed(123)
  torch.manual_seed(123)

inicializa_seeds()

## Download dos dados

In [56]:
!wget -nc http://files.fast.ai/data/aclImdb.tgz
!tar -xzf aclImdb.tgz

File ‘aclImdb.tgz’ already there; not retrieving.



## Separação em treino/validação/teste

In [57]:
import os

max_valid = 5000

def load_texts(folder):
    texts = []
    for path in os.listdir(folder):
        with open(os.path.join(folder, path)) as f:
            texts.append(f.read())
    return texts

x_train_pos = load_texts('aclImdb/train/pos')
x_train_neg = load_texts('aclImdb/train/neg')
x_test_pos = load_texts('aclImdb/test/pos')
x_test_neg = load_texts('aclImdb/test/neg')

x_train = x_train_pos + x_train_neg
x_test = x_test_pos + x_test_neg
y_train = [True] * len(x_train_pos) + [False] * len(x_train_neg)
y_test = [True] * len(x_test_pos) + [False] * len(x_test_neg)

# Embaralhamos o treino para depois fazermos a divisão treino/valid.
c = list(zip(x_train, y_train))
random.shuffle(c)
x_train, y_train = zip(*c)

x_valid = x_train[-max_valid:]
y_valid = y_train[-max_valid:]
x_train = x_train[:-max_valid]
y_train = y_train[:-max_valid]

print(len(x_train), 'amostras de treino.')
print(len(x_valid), 'amostras de desenvolvimento.')
print(len(x_test), 'amostras de teste.')

print('\n3 primeiras amostras treino:')
for x, y in zip(x_train[:3], y_train[:3]):
    print(y, x[:100])

print('\n3 últimas amostras treino:')
for x, y in zip(x_train[-3:], y_train[-3:]):
    print(y, x[:100])

print('\n3 primeiras amostras validação:')
for x, y in zip(x_test[:3], y_test[:3]):    # No código original, estava x_valid
    print(y, x[:100])

print('\n3 últimas amostras validação:')
for x, y in zip(x_valid[-3:], y_valid[-3:]):
    print(y, x[:100])

20000 amostras de treino.
5000 amostras de desenvolvimento.
25000 amostras de teste.

3 primeiras amostras treino:
False After some internet surfing, I found the "Homefront" series on DVD at ioffer.com. Before anyone gets
False Think "stage play". This is worth seeing once for the performances of Lionel Atwill and Dwight Frye.
True I've read a few of the reviews and I'm kinda sad that a lot of the Story seems glossed over. Its eas

3 últimas amostras treino:
False Fantastic Mr. Fox is a comedy based on the classic Roald Dahl book. Wes Anderson directs, and respec
True **Attention Spoilers**<br /><br />First of all, let me say that Rob Roy is one of the best films of 
True The 14 year-old in me is immensely happy that they're now able to make really good looking fantasy m

3 primeiras amostras validação:
True One of the best sitcoms to run on Indian television along with Dekh bhai dekh and Idhar udhar. Great
True Iron Eagle may not be the most believable film plot-wise, but the characte

# Tokenizador e encoder

In [58]:
## Função gerada com ajuda do ChatGPT
import re

def tokenizar(frase):
    # return frase.split() # Usar esse return para uma tokenização simples (apenas split)
    # Converter a frase para minúsculo
    frase = frase.lower()
    # Remover caracteres especiais e pontuação usando expressões regulares
    frase = re.sub(r'[^\w\s]', '', frase)
    # Dividir a frase em palavras
    palavras = frase.split()
    return palavras

# Exemplo de uso
frase = "Olá! Como você está? Eu estou bem, obrigado."
palavras = tokenizar(frase)
print(palavras)

['olá', 'como', 'você', 'está', 'eu', 'estou', 'bem', 'obrigado']


In [59]:
from collections import Counter

idx_amostras = list(range(len(x_train)))

counter_palavras = Counter()
for review in x_train:
    # Com tokenizador:
    counter_palavras.update(tokenizar(review))

# Cria um vocabulário com os tokens mais frequentes
most_frequent_words = ["<UNK>"] + sorted(counter_palavras, key=counter_palavras.get, reverse=True)[:MAX_VOCAB_SIZE-1]
vocab = {word: i for i, word in enumerate(most_frequent_words)} # word é indexado de 0 (UNK) até MAX_VOCAB_SIZE
VOCAB_SIZE = len(vocab.keys()) # Se MAX_VOCAB_SIZE for muito grande, é possível que o vocabulário não tenha palavras suficientes. Então ajusta pro tamanho correto

In [60]:
most_frequent_words[0:10]

['<UNK>', 'the', 'and', 'a', 'of', 'to', 'is', 'in', 'it', 'i']

In [61]:
def encode_sentence(sentence, vocab):
  return [vocab.get(word, 0) for word in tokenizar(sentence)] # 0 for OOV

encode_sentence("I like Pizza PALAVRAINEXISTENTE", vocab)

[9, 38, 8134, 0]

# Definição da classe Dataset

In [62]:
from torch.utils.data import Dataset, DataLoader
from typing import List

class ImdbDataset(Dataset):
  def __init__(self, x_data: List[str], y_data: List[bool], vocab) -> None:
    self.vocab = vocab

    tamanho_vocab = len(vocab.keys()) # isso dá igual VOCAB_SIZE. Pra não pegar da variável global

    # Como as bases cabem em memória, faz cache
    self.x = []
    for review in x_data:
        x = torch.zeros(tamanho_vocab)
        x[encode_sentence(review, self.vocab)] = 1
        self.x.append(x)

    self.y = None if y_data is None else [torch.tensor(1 if y_is_true else 0) for y_is_true in y_data]

  def __len__(self):
    return len(self.y)

  def __getitem__(self, idx):
    return self.x[idx], self.y[idx]

In [63]:
# Checa a implementação do dataset:

x_temp, y_temp = x_train[:10], y_train[0:10]
dataset_temp = ImdbDataset(x_temp, y_temp, vocab)

for i in range(len(x_temp)):
  encoded_x = encode_sentence(x_temp[i], vocab)
  x = torch.zeros(VOCAB_SIZE)
  x[encoded_x] = 1
  print(torch.all(x == dataset_temp[i][0]))
  print(torch.tensor(y_temp[i]) == dataset_temp[i][1])


tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)
tensor(True)


# DataLoader

In [64]:
%%time
batch_size = 128
# define dataloaders
train_data = ImdbDataset(x_train, y_train, vocab)
test_data = ImdbDataset(x_test, y_test, vocab)
val_data = ImdbDataset(x_valid, y_valid, vocab)

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False)

CPU times: user 9.78 s, sys: 1.17 s, total: 10.9 s
Wall time: 11.5 s


# Implementação camada LoRA

Verificando qual o operador mais eficiente (torch.matmul, torch.mm, @):

In [88]:
import torch.nn as nn
import time

n_vezes = 20000 if device.type == 'cuda' else 2000

temp_A = nn.Parameter(torch.Tensor(200, 1), requires_grad=False)
temp_B = nn.Parameter(torch.Tensor(1, 3000), requires_grad=False)

nn.init.normal_(temp_A, mean=0.0)
nn.init.zeros_(temp_B)

start_time = time.time()
for i in range(n_vezes):
  x = torch.matmul(temp_A, temp_B)
end_time = time.time()
print(f'matmul: {end_time - start_time}')

start_time = time.time()
for i in range(n_vezes):
  x = torch.mm(temp_A, temp_B)
end_time = time.time()
print(f'mm: {end_time - start_time}')

start_time = time.time()
for i in range(n_vezes):
  x = temp_A @ temp_B
end_time = time.time()
print(f'@: {end_time - start_time}')

matmul: 2.756267786026001
mm: 2.6500258445739746
@: 2.6752097606658936


In [66]:
import torch
import torch.nn as nn
import math

class LoRALayer(nn.Module):
  def inicializa_pesos_linear(self):
    # Parâmetros originais da camada linear
    # Note que o nn.Parameter está no formato output_dim x input_dim,
    # e não input_dim x output_dim como era de se esperar
    # Isso é feito pois o PyTorch implementa os pesos da camada linear
    # dessa forma. Em vez de calcular x*A + b, ele faz x * A_T + b,
    # onde _T é a transposta. Por isso é necessário inverter isso
    self.weight = nn.Parameter(torch.Tensor(self.output_dim, self.input_dim))
    self.bias = nn.Parameter(torch.Tensor(self.output_dim))
    nn.init.xavier_normal_(self.weight)
    nn.init.uniform_(self.bias, 0, 0)

  def inicializa_pesos_LoRA(self):
    # Parâmetros LoRA
    self.A = nn.Parameter(torch.Tensor(self.output_dim, self.rank), requires_grad=self.use_lora)
    self.B = nn.Parameter(torch.Tensor(self.rank, self.input_dim), requires_grad=self.use_lora)

    nn.init.normal_(self.A, mean=0.0)
    nn.init.zeros_(self.B)

  def __init__(self, input_dim, output_dim, rank, alpha=1.0, use_lora=False):
    super(LoRALayer, self).__init__()
    self.input_dim = input_dim
    self.output_dim = output_dim
    self.rank = rank
    self.alpha = alpha
    self.use_lora = use_lora

    self.scale = self.alpha/self.rank

    self.inicializa_pesos_linear()
    self.inicializa_pesos_LoRA()

  def forward(self, x):
    # Se estiver usando LoRA, é necessário considerar os pesos modificados
    if self.use_lora:
      delta_w = self.scale * torch.mm(self.A, self.B)
      modified_weight = self.weight + delta_w
      return nn.functional.linear(x, modified_weight, self.bias)
    else:
      # Caso contrário, usa os pesos originais
      return nn.functional.linear(x, self.weight, self.bias)
      # Obs.: O controle dos gradientes é feito dentro do toggle_lora


  def toggle_lora(self, use_lora=False):
    self.use_lora = use_lora
    # Desabilita os gradientes de weight e bias se estiver usando a camada LoRA
    self.weight.requires_grad = not self.use_lora
    self.bias.requires_grad = not self.use_lora
    # Habilita os gradientes de A e B se estiver usando a camada LoRA
    self.A.requires_grad = self.use_lora
    self.B.requires_grad = self.use_lora

  def desabilita_lora_e_transfere_para_modelo(self):
    # A ideia desse método é desabilitar o LoRA e alterar o peso original
    # para considerar o treinamento usando LoRA
    delta_w = self.scale * torch.mm(self.A, self.B)
    self.weight.data += delta_w.data

    self.toggle_lora(False)
    self.inicializa_pesos_LoRA()

In [67]:
# Testando as dimensões
# Pega um batch e passa o primeiro conjunto de inputs numa camada LoRA ativada:
batch_teste = next(iter(test_loader))
camada_lora = LoRALayer(VOCAB_SIZE, HIDDEN_SIZE, rank=10, use_lora=True)
o1 = camada_lora(batch_teste[0][0])
print('Cria uma camada LoRA e ativa o uso da camada (B é inicializado com zeros)')
print('Faz o forward usando a camada => o1\n')

# Desativa a camada LoRA e passa de novo o mesmo input:
camada_lora.toggle_lora(False)
o2 = camada_lora(batch_teste[0][0])
print('Desativa o uso da camada LoRA (faz a conta só com os pesos normais)')
print('Faz o forward => o2')
print(f'Testa de o1 == o2. Esperado: True. Resultado: {torch.all(o1 == o2)}\n')

# Na inicialização, B é tudo zero.
# Altera diretamente o B com dados aleatórios para que a camada LoRA mude o resultado
# Entretanto, mantém a camada LoRA desativada
nn.init.normal_(camada_lora.B, mean=0.0)
camada_lora.toggle_lora(False)
o3 = camada_lora(batch_teste[0][0])
print('Altera B para algum valor aleatório e mantém a camada LoRA desativada')
print('Faz o forward => o3')
print(f'Testa de o2 == o3. Esperado: True. Resultado: {torch.all(o2 == o3)}\n')

# Agora abilita a camada LoRA. Mantém B populado com algum valor
camada_lora.toggle_lora(True)
o4 = camada_lora(batch_teste[0][0])
print('Mantém B com valores e ativa a camada LoRA')
print('Faz o forward => o4')
print(f'Testa de o2 == o4. Esperado: False. Resultado: {torch.all(o2 == o4)}\n')

# Agora desabilita a camada LoRA transferindo os resultados para os pesos originais
# e calcula novamente
camada_lora.desabilita_lora_e_transfere_para_modelo()
o5 = camada_lora(batch_teste[0][0])
print('Desabilita a camada LoRA, transferindo os pesos para a camada linear')
print('Faz o forward => o5')
print(f'Testa de o4 == o5. Esperado: True. Resultado: {torch.all(o4 == o5)}. Dif máxima: {torch.max((o4-o5).abs())}\n')

# Gera qualquer coisa na camada LoRA, mas não habilita ela. Não tem que fazer diferença
nn.init.normal_(camada_lora.B, mean=0)
o6 = camada_lora(batch_teste[0][0])
print('Gera qualquer coisa na camada LoRA, mas não habilita ela')
print('Faz o forward => o6')
print(f'Testa de o5 == o6. Esperado: True. Resultado: {torch.all(o5 == o6)}\n')

# Agora habilita a camada LoRA sem inicializá-la. Tem lixo lá por conta da
# inicialização dos pesos de B com dados (teste anterior)
camada_lora.toggle_lora(True)
o7 = camada_lora(batch_teste[0][0])
print('Habilita a camada LoRA mantendo lixo em B')
print('Faz o forward => o7')
print(f'Testa de o6 == o7. Esperado: False. Resultado: {torch.all(o6 == o7)}\n')

Cria uma camada LoRA e ativa o uso da camada (B é inicializado com zeros)
Faz o forward usando a camada => o1

Desativa o uso da camada LoRA (faz a conta só com os pesos normais)
Faz o forward => o2
Testa de o1 == o2. Esperado: True. Resultado: True

Altera B para algum valor aleatório e mantém a camada LoRA desativada
Faz o forward => o3
Testa de o2 == o3. Esperado: True. Resultado: True

Mantém B com valores e ativa a camada LoRA
Faz o forward => o4
Testa de o2 == o4. Esperado: False. Resultado: False

Desabilita a camada LoRA, transferindo os pesos para a camada linear
Faz o forward => o5
Testa de o4 == o5. Esperado: True. Resultado: True. Dif máxima: 0.0

Gera qualquer coisa na camada LoRA, mas não habilita ela
Faz o forward => o6
Testa de o5 == o6. Esperado: True. Resultado: True

Habilita a camada LoRA mantendo lixo em B
Faz o forward => o7
Testa de o6 == o7. Esperado: False. Resultado: False



# Modelo

In [68]:
import torch.nn as nn
import torch.optim as optim

class OneHotMLP(nn.Module):
  def __init__(self, vocab_size, hidden_size, rank, alpha, n_logitos=2):
    super(OneHotMLP, self).__init__()

    self.n_logitos = n_logitos

    #self.fc1 = nn.Linear(vocab_size, hidden_size)
    # No início do treinamento é feito sem o uso da camada LoRA, por isso começa com use_lora=False
    self.fc1 = LoRALayer(vocab_size, hidden_size, rank, alpha, use_lora=False)
    self.fc2 = nn.Linear(hidden_size, n_logitos)

    self.relu = nn.ReLU()

  def forward(self, x):
    # x tem tamanho [B, VOCAB]

    # Após a primeira camada, o tamanho será [B, HIDDEN_SIZE]
    o = self.fc1(x)

    # Após ReLU, o tamanho continua [B, HIDDEN_SIZE]
    o = self.relu(o)

    # Após a segunda camada, o tamanho será [B, n_logitos]
    o = self.fc2(o)

    return o

  def desabilita_lora_e_transfere_para_modelo(self):
    self.fc1.desabilita_lora_e_transfere_para_modelo()

  def toggle_lora(self, use_lora):
    self.fc1.toggle_lora(use_lora)


# Laço de treinamento

In [69]:
def calcula_loss_e_acuracia(model, loader):
  criterion = nn.CrossEntropyLoss()

  # Garante que nenhum gradiente seja calculado
  with torch.no_grad():
    # Coloca o modelo no modo de avaliação (não treinamento)
    model.eval()

    total_loss = 0.0
    total_samples = 0
    total_predicoes_corretas = 0.0

    for inputs, labels in loader:
      inputs = inputs.to(device)
      labels = labels.to(device)
      # Forward pass
      outputs = model(inputs)
      # Calcula a perda
      probabilities = nn.functional.softmax(outputs, dim=1)
      loss = criterion(probabilities, labels)
      # Conta as predições corretas (para o cálculo da acurácia)
      _, predicoes = torch.max(probabilities, 1)
      total_predicoes_corretas += (predicoes == labels).sum().item()

      # Acumula a perda e o número total de amostras
      total_loss += loss.item() * inputs.size(0)
      total_samples += inputs.size(0)

  loss = total_loss / total_samples
  acuracia = total_predicoes_corretas / total_samples

  return loss, acuracia

def print_loss_acuracia(msg, model, loader):
  loss, acc = calcula_loss_e_acuracia(model, loader)
  print(f"{msg}. Loss: {loss}. Acc: {acc}")
  return loss, acc

In [70]:
# Testa o cálculo da loss em um dataloader. Ele deve ser aproximadamente ln(2):
import math

# Model instantiation
model = OneHotMLP(VOCAB_SIZE, HIDDEN_SIZE, RANK, ALPHA, 2).to(device)

print(f"Valor esperado da loss: {math.log(2)}")
print_loss_acuracia("Antes de treinar o modelo [VAL]: ", model, val_loader)

Valor esperado da loss: 0.6931471805599453
Antes de treinar o modelo [VAL]: . Loss: 0.6935397658348084. Acc: 0.4912


(0.6935397658348084, 0.4912)

In [71]:
temp1 = list(filter(lambda p: p.requires_grad, model.parameters()))
temp2 = list(filter(lambda p: True, model.parameters()))

In [72]:
from tqdm import tqdm
import time

def train_model(model, lr=LR, num_epochs=NUM_EPOCHS):
  model = model.to(device)
  # optimizer = optim.SGD(model.parameters(), lr=lr)
  # Usa só os parâmetros que devem ser atualizados
  optimizer = optim.SGD(filter(lambda p: p.requires_grad, model.parameters()), lr=lr)
  criterion = nn.CrossEntropyLoss()

  total_parametros_treinaveis = sum(p.numel() for p in model.parameters() if p.requires_grad)
  total_parametros = sum(p.numel() for p in model.parameters())
  print(f'Total de parâmetros treináveis: {total_parametros_treinaveis}/{total_parametros}')
  print_loss_acuracia("[TRAIN] Antes de iniciar o treinamento", model, train_loader)
  # Training loop
  total_tempo_treinamento = 0
  for epoch in range(1, num_epochs+1):
    print(f'************************ EPOCH {epoch} ************************')
    start_time = time.time()  # Start time of the epoch
    model.train()
    for inputs, labels in tqdm(train_loader, desc=f"Epoch: {epoch}"):
      inputs = inputs.to(device)
      labels = labels.to(device)
      # Forward pass
      outputs = model(inputs)
      loss = criterion(nn.functional.softmax(outputs, dim=1), labels)
      # Backward and optimize
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

    end_time = time.time()  # End time of the epoch
    epoch_duration = end_time - start_time  # Duration of epoch
    total_tempo_treinamento += epoch_duration

    print(f'Resultados da época {epoch}')
    print(f"Elapsed Time: {epoch_duration:.2f} sec")
    print_loss_acuracia('[TRAIN]', model, train_loader)
    print_loss_acuracia('[VAL]', model, val_loader)
  print('*'*20)
  print(f'Duração média durante o treino, por época: {total_tempo_treinamento/num_epochs:.2f}')

In [73]:
inicializa_seeds()
model = OneHotMLP(VOCAB_SIZE, HIDDEN_SIZE, RANK, ALPHA, 2)
train_model(model, lr=LR, num_epochs=NUM_EPOCHS)

Total de parâmetros treináveis: 4000602/4020802
[TRAIN] Antes de iniciar o treinamento. Loss: 0.6934093474388122. Acc: 0.50345
************************ EPOCH 1 ************************


Epoch: 1: 100%|██████████| 157/157 [00:00<00:00, 162.79it/s]


Resultados da época 1
Elapsed Time: 0.97 sec
[TRAIN]. Loss: 0.5483880427360535. Acc: 0.8184
[VAL]. Loss: 0.5536890678405761. Acc: 0.809
************************ EPOCH 2 ************************


Epoch: 2: 100%|██████████| 157/157 [00:00<00:00, 166.79it/s]


Resultados da época 2
Elapsed Time: 0.95 sec
[TRAIN]. Loss: 0.49788768944740297. Acc: 0.8201
[VAL]. Loss: 0.505357151556015. Acc: 0.8122
************************ EPOCH 3 ************************


...

Resultados da época 5
Elapsed Time: 0.97 sec
[TRAIN]. Loss: 0.41854159569740296. Acc: 0.90735
[VAL]. Loss: 0.44328261017799375. Acc: 0.8702
************************ EPOCH 6 ************************


Epoch: 6: 100%|██████████| 157/157 [00:00<00:00, 164.91it/s]


Resultados da época 6
Elapsed Time: 0.96 sec
[TRAIN]. Loss: 0.409752996635437. Acc: 0.917
[VAL]. Loss: 0.4387821786403656. Acc: 0.875
************************ EPOCH 7 ************************


Epoch: 7: 100%|██████████| 157/157 [00:00<00:00, 164.37it/s]


Resultados da época 7
Elapsed Time: 0.96 sec
[TRAIN]. Loss: 0.4036974662780762. Acc: 0.92205
[VAL]. Loss: 0.4357549529075623. Acc: 0.8794
********************
Duração média durante o treino, por época: 0.98


Com esses resultados, dá pra imaginar que é possível ter aproximadamente 87~88% de acurácia na base de testes:

In [74]:
print_loss_acuracia("[TESTE]", model, test_loader)

[TESTE]. Loss: 0.43823531591415404. Acc: 0.87756


(0.43823531591415404, 0.87756)

## Testando LoRA

Vamos fazer o seguinte:

1. Reiniciar a seed e reinicializar o modelo.
2. Simular ele completo com apenas uma época => Esse será o modelo pré-treinado (tem uns 78% de acurácia no conjunto de validação => esse valor inicial irá mudar dependendo da inicialização do modelo, mas basta que ele seja alguns pontos abaixo do target de 87-88% pra testarmos)
3. Ativar as camadas LoRA.
4. Simular novamente e verificar melhorias no tempo e se é possível chegar numa acurácia próxima a de treinar o modelo completo.

PASSOS 1 e 2. Reiniciar a seed, reinicializar o modelo, simular com uma época

In [75]:
inicializa_seeds()
model = OneHotMLP(VOCAB_SIZE, HIDDEN_SIZE, RANK, ALPHA, 2)
train_model(model, lr=LR, num_epochs=1)

Total de parâmetros treináveis: 4000602/4020802
[TRAIN] Antes de iniciar o treinamento. Loss: 0.6934093474388122. Acc: 0.50345
************************ EPOCH 1 ************************


Epoch: 1: 100%|██████████| 157/157 [00:00<00:00, 158.92it/s]


Resultados da época 1
Elapsed Time: 1.00 sec
[TRAIN]. Loss: 0.5483880427360535. Acc: 0.8184
[VAL]. Loss: 0.5536890678405761. Acc: 0.809
********************
Duração média durante o treino, por época: 1.00


In [76]:
print_loss_acuracia("[TESTE]", model, test_loader)

[TESTE]. Loss: 0.5552642659378052. Acc: 0.80752


(0.5552642659378052, 0.80752)

PASSOS 3 e 4. Ativar as camadas LoRA e simular

Mas antes de ativar, vamos ver como estão os pesos da camada fc1 do modelo (weight e bias). Depois de treinar os pesos devem ficar iguais. Vamos checar também as matrizes A e B.

In [77]:
# Guarda para comparar depois
fc1_0 = model.fc1.weight[0, 1:10].detach().clone()
fc1_1 = model.fc1.weight[1, 1:10].detach().clone()
b_0 = model.fc1.bias[1:10].detach().clone()

print('w[0, 1:10]', model.fc1.weight[0, 1:10], '\n')
print('w[1, 1:10]', model.fc1.weight[1, 1:10], '\n')
print('b[1:10]', model.fc1.bias[1:10], '\n')
print('A[0: 1:10]', model.fc1.A[0, 1:10], '\n') # Inicialização gausiana, média 0
print('A[1: 1:10]', model.fc1.A[1, 1:10], '\n') # Inicialização gausiana, média 0
print('B[0: 1:10]', model.fc1.B[0, 1:10], '\n') # Tem que ter só zeros aqui

w[0, 1:10] tensor([-0.0016, -0.0029, -0.0058,  0.0036,  0.0067, -0.0021, -0.0037,  0.0078,
        -0.0117], device='cuda:0', grad_fn=<SliceBackward0>) 

w[1, 1:10] tensor([ 0.0043,  0.0160,  0.0090, -0.0186,  0.0127, -0.0036,  0.0069, -0.0049,
        -0.0183], device='cuda:0', grad_fn=<SliceBackward0>) 

b[1:10] tensor([ 0.0005, -0.0009,  0.0034, -0.0020,  0.0026,  0.0037,  0.0009,  0.0024,
        -0.0006], device='cuda:0', grad_fn=<SliceBackward0>) 

A[0: 1:10] tensor([], device='cuda:0') 

A[1: 1:10] tensor([], device='cuda:0') 

B[0: 1:10] tensor([0., 0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0') 



In [78]:
model.toggle_lora(True)
train_model(model, lr=LR, num_epochs=NUM_EPOCHS_LORA)

Total de parâmetros treináveis: 20602/4020802
[TRAIN] Antes de iniciar o treinamento. Loss: 0.548388037109375. Acc: 0.8184
************************ EPOCH 1 ************************


Epoch: 1: 100%|██████████| 157/157 [00:00<00:00, 164.02it/s]


Resultados da época 1
Elapsed Time: 0.96 sec
[TRAIN]. Loss: 0.475860825920105. Acc: 0.8482
[VAL]. Loss: 0.48563785510063173. Acc: 0.838
************************ EPOCH 2 ************************


Epoch: 2: 100%|██████████| 157/157 [00:00<00:00, 164.35it/s]


Resultados da época 2
Elapsed Time: 0.96 sec
[TRAIN]. Loss: 0.43129615206718447. Acc: 0.88355
[VAL]. Loss: 0.45073085746765135. Acc: 0.8574
************************ EPOCH 3 ************************


...

Resultados da época 8
Elapsed Time: 0.96 sec
[TRAIN]. Loss: 0.48127295837402345. Acc: 0.8187
[VAL]. Loss: 0.5025337201595307. Acc: 0.8014
************************ EPOCH 9 ************************


Epoch: 9: 100%|██████████| 157/157 [00:00<00:00, 157.78it/s]


Resultados da época 9
Elapsed Time: 1.00 sec
[TRAIN]. Loss: 0.401526517534256. Acc: 0.90645
[VAL]. Loss: 0.4480236240386963. Acc: 0.8588
************************ EPOCH 10 ************************


Epoch: 10: 100%|██████████| 157/157 [00:00<00:00, 157.38it/s]


Resultados da época 10
Elapsed Time: 1.01 sec
[TRAIN]. Loss: 0.3656315544128418. Acc: 0.9484
[VAL]. Loss: 0.43123267207145694. Acc: 0.8758
********************
Duração média durante o treino, por época: 1.00


In [79]:
print_loss_acuracia("[TESTE]", model, test_loader)

[TESTE]. Loss: 0.433113478307724. Acc: 0.87492


(0.433113478307724, 0.87492)

Checa se os parâmetros weight e bias da camada LoRA mudaram depois do fine-tuning ou se apenas ajustaram as camadas A e B da camada LoRA:

In [80]:
print(torch.all(fc1_0 == model.fc1.weight[0, 1:10])) # Tem que ser True
print(torch.all(fc1_1 == model.fc1.weight[1, 1:10])) # Tem que ser True
print(torch.all(b_0 == model.fc1.bias[1:10])) # Tem que ser True

print('w[0, 1:10]', model.fc1.weight[0, 1:10], '\n')
print('w[1, 1:10]', model.fc1.weight[1, 1:10], '\n')
print('b[1:10]', model.fc1.bias[1:10], '\n')
print('A[0: 1:10]', model.fc1.A[0, 1:10], '\n') # Nesse ponto essa matriz já foi treinada, tem que ter alguma coisa diferente
print('A[1: 1:10]', model.fc1.A[1, 1:10], '\n') # Nesse ponto essa matriz já foi treinada, tem que ter alguma coisa diferente
print('B[0: 1:10]', model.fc1.B[0, 1:10], '\n') # Nesse ponto essa matriz já foi treinada, não pode continuar zerada

tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
w[0, 1:10] tensor([-0.0016, -0.0029, -0.0058,  0.0036,  0.0067, -0.0021, -0.0037,  0.0078,
        -0.0117], device='cuda:0') 

w[1, 1:10] tensor([ 0.0043,  0.0160,  0.0090, -0.0186,  0.0127, -0.0036,  0.0069, -0.0049,
        -0.0183], device='cuda:0') 

b[1:10] tensor([ 0.0005, -0.0009,  0.0034, -0.0020,  0.0026,  0.0037,  0.0009,  0.0024,
        -0.0006], device='cuda:0') 

A[0: 1:10] tensor([], device='cuda:0', grad_fn=<SliceBackward0>) 

A[1: 1:10] tensor([], device='cuda:0', grad_fn=<SliceBackward0>) 

B[0: 1:10] tensor([-3.5144e-02, -5.4687e-03,  5.3679e-03,  6.3311e-05,  7.9117e-03,
        -3.5592e-02,  4.2383e-02, -1.8123e-02, -2.9819e-02], device='cuda:0',
       grad_fn=<SliceBackward0>) 



Transfere a camada LoRA para o modelo (altera weight para scale * A * B):

In [81]:
model.desabilita_lora_e_transfere_para_modelo()

In [82]:
print(torch.all(fc1_0 == model.fc1.weight[0, 1:10])) # Tem que ser False
print(torch.all(fc1_1 == model.fc1.weight[1, 1:10])) # Tem que ser False
print(torch.all(b_0 == model.fc1.bias[1:10])) # Tem que ser True

print('w[0, 1:10]', model.fc1.weight[0, 1:10], '\n')
print('w[1, 1:10]', model.fc1.weight[1, 1:10], '\n')
print('b[1:10]', model.fc1.bias[1:10], '\n')
print('A[0: 1:10]', model.fc1.A[0, 1:10], '\n')
print('A[1: 1:10]', model.fc1.A[1, 1:10], '\n')
print('B[0: 1:10]', model.fc1.B[0, 1:10], '\n')

tensor(False, device='cuda:0')
tensor(False, device='cuda:0')
tensor(True, device='cuda:0')
w[0, 1:10] tensor([ 0.0265,  0.0014, -0.0101,  0.0036,  0.0004,  0.0264, -0.0376,  0.0223,
         0.0122], device='cuda:0', grad_fn=<SliceBackward0>) 

w[1, 1:10] tensor([-0.0787,  0.0030,  0.0216, -0.0185,  0.0313, -0.0877,  0.1070, -0.0477,
        -0.0887], device='cuda:0', grad_fn=<SliceBackward0>) 

b[1:10] tensor([ 0.0005, -0.0009,  0.0034, -0.0020,  0.0026,  0.0037,  0.0009,  0.0024,
        -0.0006], device='cuda:0', grad_fn=<SliceBackward0>) 

A[0: 1:10] tensor([]) 

A[1: 1:10] tensor([]) 

B[0: 1:10] tensor([0., 0., 0., 0., 0., 0., 0., 0., 0.]) 



Agora que já jogou os dados de A e B para weight, calcula novamente a acurácia (deve estar igual o último cálculo)

In [83]:
print_loss_acuracia("[TESTE]", model, test_loader)

[TESTE]. Loss: 0.433113478307724. Acc: 0.87492


(0.433113478307724, 0.87492)